# Most likely not at all used

also try to make a linear model to see if it is better!

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from warnings import filterwarnings
filterwarnings('ignore')
import joblib



In [2]:
data = pd.read_csv('output.csv')

X = data[['X', 'Y', 'Z']]
y = data[['PixelX', 'PixelY']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [3]:
model = MLPRegressor(hidden_layer_sizes=(10,10), max_iter=1, warm_start=True)

train_losses = []
test_losses = []

In [ ]:
for i in range(2500):  
    model.fit(X_train_scaled, y_train)
    train_pred = model.predict(X_train_scaled)
    test_pred = model.predict(X_test_scaled)
    
    train_loss = mean_squared_error(y_train, train_pred)
    test_loss = mean_squared_error(y_test, test_pred)
    
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    if i%100 == 1:
        print('Loss:', train_loss, test_loss)

In [ ]:
plt.plot(train_losses, label='Train Loss')
plt.plot(test_losses, label='Test Loss')
plt.title('Train and Test Loss Curve')
plt.xlabel('Iterations')
plt.ylabel('Mean Squared Error')
plt.legend()
plt.show()

In [ ]:
joblib.dump(model, 'NN_model_xyz_to_XY.pkl')

In [7]:
import joblib
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from warnings import filterwarnings
scaler = StandardScaler()


In [ ]:
loaded_model = joblib.load('NN_model_xyz_to_XY.pkl')
sample_input = [[0.5, -1.2, 3.4]]
sample_input_scaled = scaler.transform(sample_input)
prediction = loaded_model.predict(sample_input_scaled)

print("Predicted PixelX, PixelY:", prediction)


In [ ]:
import joblib
import numpy as np

# Load the model
loaded_model = joblib.load('NN_model_xyz_to_XY.pkl')

# Define the input data (X, Y, Z values)
input_data = np.array([
    [-8.179, 1.226, 26.594],
    [-8.139, 2.576, 26.746],
    [-8.289, 4.123, 27.328],
    [-8.283, 5.653, 27.431],
    [-8.238, 7.08, 27.312],
    [-8.256, 8.501, 27.454],
    [-7.866, 9.739, 26.589],
    [-8.279, -10.466, 26.065]
])

# Original PixelX, PixelY values for comparison
original_outputs = np.array([
    [54, 560],
    [52, 624],
    [52, 682],
    [49, 757],
    [54, 810],
    [61, 879],
    [59, 938],
    [57, 51]
])

# Scale the input data
input_data_scaled = scaler.transform(input_data)

# Make predictions
predictions = loaded_model.predict(input_data_scaled)

# Print results
print("Predicted vs Original PixelX, PixelY values:")
for i, (pred, orig) in enumerate(zip(predictions, original_outputs)):
    print(f"Point {i+1}: Predicted {pred}, Original {orig}")


In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from warnings import filterwarnings
import joblib

filterwarnings('ignore')

class CoordinatePredictor:
    def __init__(self):
        self.scaler = StandardScaler()
        self.nn_model = None
        self.linear_model = None
        
    def prepare_data(self, data_path):
        """Load and prepare data for training"""
        data = pd.read_csv(data_path)
        X = data[['X', 'Y', 'Z']]
        y = data[['PixelX', 'PixelY']]
        
        return train_test_split(X, y, test_size=0.2, random_state=42)
    
    def train_neural_network(self, X_train, X_test, y_train, y_test, 
                           hidden_layers=(10,10), max_iterations=10000):
        """Train neural network model and return losses"""
        X_train_scaled = self.scaler.fit_transform(X_train)
        X_test_scaled = self.scaler.transform(X_test)
        
        self.nn_model = MLPRegressor(
            hidden_layer_sizes=hidden_layers,
            max_iter=1,
            warm_start=True,
            random_state=42
        )
        
        train_losses = []
        test_losses = []
        
        for i in range(max_iterations):
            self.nn_model.fit(X_train_scaled, y_train)
            
            train_pred = self.nn_model.predict(X_train_scaled)
            test_pred = self.nn_model.predict(X_test_scaled)
            
            train_loss = mean_squared_error(y_train, train_pred)
            test_loss = mean_squared_error(y_test, test_pred)
            
            train_losses.append(train_loss)
            test_losses.append(test_loss)
            
            if i % 100 == 0:
                print(f'Iteration {i}: Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}')
        
        return train_losses, test_losses
    
    def train_linear_model(self, X_train, X_test, y_train, y_test):
        """Train linear regression model and return metrics"""
        X_train_scaled = self.scaler.transform(X_train)
        X_test_scaled = self.scaler.transform(X_test)
        
        self.linear_model = LinearRegression()
        self.linear_model.fit(X_train_scaled, y_train)
        
        train_pred = self.linear_model.predict(X_train_scaled)
        test_pred = self.linear_model.predict(X_test_scaled)
        
        train_loss = mean_squared_error(y_train, train_pred)
        test_loss = mean_squared_error(y_test, test_pred)
        r2 = r2_score(y_test, test_pred)
        
        return train_loss, test_loss, r2
    
    def plot_nn_losses(self, train_losses, test_losses):
        """Plot neural network training curves"""
        plt.figure(figsize=(10, 6))
        plt.plot(train_losses, label='Train Loss')
        plt.plot(test_losses, label='Test Loss')
        plt.title('Neural Network Training and Test Loss Curves')
        plt.xlabel('Iterations')
        plt.ylabel('Mean Squared Error')
        plt.legend()
        plt.grid(True)
        plt.show()
    
    def save_models(self, nn_path='NN_model.pkl', linear_path='Linear_model.pkl'):
        """Save both models to disk"""
        if self.nn_model:
            joblib.dump(self.nn_model, nn_path)
        if self.linear_model:
            joblib.dump(self.linear_model, linear_path)
    
    def load_models(self, nn_path='NN_model.pkl', linear_path='Linear_model.pkl'):
        """Load models from disk"""
        self.nn_model = joblib.load(nn_path)
        self.linear_model = joblib.load(linear_path)
    
    def predict(self, input_data, model_type='linear'):
        """Make predictions using specified model"""
        input_scaled = self.scaler.transform(input_data)
        
        if model_type.lower() == 'nn' and self.nn_model:
            return self.nn_model.predict(input_scaled)
        elif model_type.lower() == 'linear' and self.linear_model:
            return self.linear_model.predict(input_scaled)
        else:
            raise ValueError("Invalid model type or model not trained")



In [11]:
predictor = CoordinatePredictor()

# Prepare data
X_train, X_test, y_train, y_test = predictor.prepare_data('output.csv')

# Train and evaluate neural network
print("\nTraining Neural Network...")
train_losses, test_losses = predictor.train_neural_network(X_train, X_test, y_train, y_test)
predictor.plot_nn_losses(train_losses, test_losses)

# Train and evaluate linear model
print("\nTraining Linear Model...")
train_loss, test_loss, r2 = predictor.train_linear_model(X_train, X_test, y_train, y_test)
print(f"Linear Model Metrics:")
print(f"Train MSE: {train_loss:.4f}")
print(f"Test MSE: {test_loss:.4f}")
print(f"R² Score: {r2:.4f}")

# Save models
predictor.save_models()

# Example prediction
sample_input = np.array([[-8.179, 1.226, 26.594]])
nn_pred = predictor.predict(sample_input, 'nn')
linear_pred = predictor.predict(sample_input, 'linear')

print("\nPredictions for sample input:")
print(f"Neural Network: {nn_pred}")
print(f"Linear Model: {linear_pred}")



Training Neural Network...
Iteration 0: Train Loss: 313988.1599, Test Loss: 359381.1168
Iteration 100: Train Loss: 313226.1526, Test Loss: 358418.4198
Iteration 200: Train Loss: 311644.5138, Test Loss: 356464.2582
Iteration 300: Train Loss: 308633.4562, Test Loss: 352827.9213
Iteration 400: Train Loss: 303493.4731, Test Loss: 346612.4255
Iteration 500: Train Loss: 295583.0949, Test Loss: 337032.3448
Iteration 600: Train Loss: 284410.1235, Test Loss: 323510.9186
Iteration 700: Train Loss: 269600.3924, Test Loss: 305559.7760
Iteration 800: Train Loss: 250930.6226, Test Loss: 282894.3939
Iteration 900: Train Loss: 228503.0758, Test Loss: 255668.9136
Iteration 1000: Train Loss: 202698.5736, Test Loss: 224347.6972
Iteration 1100: Train Loss: 174324.4740, Test Loss: 189807.4578
Iteration 1200: Train Loss: 144728.1408, Test Loss: 153758.9364
Iteration 1300: Train Loss: 115595.8300, Test Loss: 118709.5731
Iteration 1400: Train Loss: 88573.7552, Test Loss: 86754.5262
Iteration 1500: Train Loss

KeyboardInterrupt: 

In [17]:
import joblib
import numpy as np
from sklearn.preprocessing import StandardScaler

# Define the actual training range (min/max values from the dataset)
training_range = np.array([
    [-8.404, -10.725, 25.021],  # min values (X, Y, Z)
    [11.644, 10.137, 27.974]    # max values (X, Y, Z)
])
# Create and fit scaler with training range
scaler = StandardScaler()
scaler.fit(training_range)

# Load model
model = joblib.load('NN_model_xyz_to_XY.pkl')

# Example input (replace with your coordinates)
input_data = np.array([
    [9.975,-0.33,26.099,]
])

# Scale and predict
input_scaled = scaler.transform(input_data)
predictions = model.predict(input_scaled)
print("Predicted PixelX, PixelY:", predictions)

Predicted PixelX, PixelY: [[661.84967933 488.31360644]]


# The real one

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import joblib

# Read data
data = pd.read_csv('output.csv')

# Prepare features and targets
X = data[['X', 'Y', 'Z']]
y_pixelX = data['PixelX']
y_pixelY = data['PixelY']

# Create and fit scalers
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train models for X and Y pixel coordinates
model_pixelX = LinearRegression()
model_pixelY = LinearRegression()

model_pixelX.fit(X_scaled, y_pixelX,)
model_pixelY.fit(X_scaled, y_pixelY)

# Print R² scores
print(f"PixelX R² score: {model_pixelX.score(X_scaled, y_pixelX):.4f}")
print(f"PixelY R² score: {model_pixelY.score(X_scaled, y_pixelY):.4f}")

# Save models and scaler
joblib.dump(model_pixelX, 'model_pixelX.pkl')
joblib.dump(model_pixelY, 'model_pixelY.pkl')
joblib.dump(scaler, 'scaler.pkl')

# Optional: Print coefficients
print("\nPixelX coefficients:", model_pixelX.coef_)
print("PixelX intercept:", model_pixelX.intercept_)
print("\nPixelY coefficients:", model_pixelY.coef_)
print("PixelY intercept:", model_pixelY.intercept_)

PixelX R² score: 0.9996
PixelY R² score: 0.9996

PixelX coefficients: [273.89924623  -3.47144254  -0.80198502]
PixelX intercept: 497.85333333333335

PixelY coefficients: [  4.09447346 275.09379539  -0.7497379 ]
PixelY intercept: 497.4488888888889


In [22]:
import joblib
import numpy as np

# Load models and scaler
model_pixelX = joblib.load('model_pixelX.pkl')
model_pixelY = joblib.load('model_pixelY.pkl')
scaler = joblib.load('scaler.pkl')

input_data = np.array([
    [-6.29, -5.35, 26.0],  # Point 1
    [4.07, -5.75, 26.0],   # Point 2
    [-8.07, 5.31, 26.0],   # Point 3
    [8.04, 1.83, 26.0],    # Point 4
    [1.26, 4.12, 26.0],    # Point 5
    [7.45, 9.88, 26.0]     # Point 6
])

# Scale input
input_scaled = scaler.transform(input_data)

# Predict
pixel_x = model_pixelX.predict(input_scaled)
pixel_y = model_pixelY.predict(input_scaled)

# Print results
for i in range(len(input_data)):
    print(f"Input {i+1}: {input_data[i]}")
    print(f"Predicted Pixel X: {pixel_x[i]:.1f}, Pixel Y: {pixel_y[i]:.1f}\n")

Input 1: [-6.29 -5.35 26.  ]
Predicted Pixel X: 146.7, Pixel Y: 271.5

Input 2: [ 4.07 -5.75 26.  ]
Predicted Pixel X: 613.8, Pixel Y: 260.8

Input 3: [-8.07  5.31 26.  ]
Predicted Pixel X: 60.5, Pixel Y: 740.9

Input 4: [ 8.04  1.83 26.  ]
Predicted Pixel X: 788.5, Pixel Y: 598.1

Input 5: [ 1.26  4.12 26.  ]
Predicted Pixel X: 481.7, Pixel Y: 694.7

Input 6: [ 7.45  9.88 26.  ]
Predicted Pixel X: 757.5, Pixel Y: 953.2

